In [1]:
import torch
from torch import nn
import torchvision.models as models

model = models.resnet18(num_classes=26)  #！！！！！！remember to change this to correct number！！！
conv1 = model.conv1
model.conv1 = nn.Conv2d(
  2, conv1.out_channels, conv1.kernel_size, conv1.stride, conv1.padding,
  conv1.dilation, conv1.groups, conv1.bias)

In [2]:
import random
device = cuda0 = torch.device('cuda:0')
cpu = torch.device('cpu')


features_fn = '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/input/resnet_node_features.txt'
graph_targets_fn = '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/input/resnet_graph_targets.txt'

# magic numbers
INPUT_CHANNELS = 1
OUTPUT_CHANNELS = 26
BATCH_SIZE = 64
EPOCHS = 500 #set this to 200 - 2000
BENCHMARKING = False
random.seed = 88888888

In [3]:
# we'll need the target encoder from this code block... not sure if we need anything else
def build_resnet_datalist(node_features_fn, graph_targets_fn):
    feature_v = numpy.loadtxt(node_features_fn)
    target_v = numpy.loadtxt(graph_targets_fn,dtype=str,delimiter=",")
    
    target_encoder = sklearn.preprocessing.LabelEncoder()
    target_v = target_encoder.fit_transform(target_v)
    label_mapping = dict(zip(target_encoder.transform(target_encoder.classes_), target_encoder.classes_))
    print(label_mapping)
    print(len(target_v))
    print("----")
    print(len(feature_v))
    data_list = []
    for row_idx in range(len(feature_v)):
      x = torch.tensor(feature_v[row_idx,:],dtype=torch.float)
      x = x.reshape(2,8,491)
      y = torch.tensor([target_v[row_idx]])
      data_list.append({'x':x,'y':y})
    
    return data_list

def build_reactome_graph_loader(data_list,batch_size):

    loader = DataLoader(data_list,batch_size=batch_size,shuffle=True)

    return loader

In [4]:
import numpy
import sklearn
from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader
data_list = build_resnet_datalist(features_fn, graph_targets_fn)
random.shuffle(data_list)

{0: '"Adipose"', 1: '"Adrenal"', 2: '"Aorta"', 3: '"Bone"', 4: '"Brain"', 5: '"Eye"', 6: '"Heart"', 7: '"Intestine"', 8: '"Kidney"', 9: '"Liver"', 10: '"Lung"', 11: '"MOE"', 12: '"Mammary gland"', 13: '"Muscle"', 14: '"Nerve"', 15: '"Olfactory"', 16: '"Ovary"', 17: '"Pancreas"', 18: '"Skin"', 19: '"Sperm"', 20: '"Spinal cord"', 21: '"Stomach"', 22: '"Tendon"', 23: '"Testes"', 24: '"Uterus"', 25: '"VNO"'}
6288
----
6288


In [5]:
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

def train(loader,device):
  model.train()

  for batch in loader:  # Iterate in batches over the training dataset.
    x = batch['x'].to(device)
    y = batch['y'].to(device)
    out = model(x)  # Perform a single forward pass.
    y = torch.squeeze(y)
    loss = criterion(out, y)  # Compute the loss.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    optimizer.zero_grad()  # Clear gradients.

def test(loader,device):
  model.eval()

  correct = 0
  for batch in loader:  # Iterate in batches over the training/test dataset.
    x = batch['x'].to(device)
    y = batch['y'].to(device)
    out = model(x)  # Perform a single forward pass.
    y = torch.squeeze(y)
    loss = criterion(out, y)  # Compute the loss.
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    correct += int((pred == y).sum())  # Check against ground-truth labels.
  return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [6]:
acc_str = ''
fold = 'full_dataset'
fold_size = 911

#>>> train =              z[:fold_size * (fold - 1)] +         z[fold_size * fold:]
#train_data_list = data_list[:fold_size * (fold - 1)] + data_list[fold_size * fold:]
#>>> test =              z[fold_size * (fold - 1):fold_size * fold]
#test_data_list = data_list[fold_size * (fold - 1):fold_size * fold]
train_data_list = data_list

print(f'Number of training examples: {len(train_data_list)}')
#print(f'Number of test examples: {len(test_data_list)}')
train_data_loader = build_reactome_graph_loader(train_data_list,BATCH_SIZE)
#test_data_loader = build_reactome_graph_loader(test_data_list,BATCH_SIZE)
for epoch in range(EPOCHS):
  train(train_data_loader,device)
  train_acc = test(train_data_loader,device)
  #test_acc = test(test_data_loader,device)
  acc_str += f'{train_acc:.4f}'#',{test_acc:.4f}\n'
  print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')#', Test Acc: {test_acc:.4f}')

training_acc_fn = F"resnet_classification_acc_fold_{fold}.txt"
path = F"/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/GNN/{training_acc_fn}"
with open(path, 'w') as writefile:
    writefile.write(acc_str)
model_save_name = F"trained_pytorch_model_fold_{fold}.pt"
path = F"/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/GNN/{model_save_name}" 
torch.save(model.state_dict(), path)
print(F"model saved as {path}")

Number of training examples: 6288
Epoch: 000, Train Acc: 0.9385
Epoch: 001, Train Acc: 0.9674
Epoch: 002, Train Acc: 0.9664
Epoch: 003, Train Acc: 0.9838
Epoch: 004, Train Acc: 0.9890
Epoch: 005, Train Acc: 0.9913
Epoch: 006, Train Acc: 0.9892
Epoch: 007, Train Acc: 0.9887
Epoch: 008, Train Acc: 0.9863
Epoch: 009, Train Acc: 0.9812
Epoch: 010, Train Acc: 0.9879
Epoch: 011, Train Acc: 0.9895
Epoch: 012, Train Acc: 0.9944
Epoch: 013, Train Acc: 0.9963
Epoch: 014, Train Acc: 0.9940
Epoch: 015, Train Acc: 0.9866
Epoch: 016, Train Acc: 0.9862
Epoch: 017, Train Acc: 0.9965
Epoch: 018, Train Acc: 0.9978
Epoch: 019, Train Acc: 0.9978
Epoch: 020, Train Acc: 0.9976
Epoch: 021, Train Acc: 0.9979
Epoch: 022, Train Acc: 0.9979
Epoch: 023, Train Acc: 0.9983
Epoch: 024, Train Acc: 0.9989
Epoch: 025, Train Acc: 0.9975
Epoch: 026, Train Acc: 0.9978
Epoch: 027, Train Acc: 0.9986
Epoch: 028, Train Acc: 0.9984
Epoch: 029, Train Acc: 0.9248
Epoch: 030, Train Acc: 0.9711
Epoch: 031, Train Acc: 0.9924
Epoch:

Epoch: 272, Train Acc: 0.9992
Epoch: 273, Train Acc: 0.9992
Epoch: 274, Train Acc: 0.9992
Epoch: 275, Train Acc: 0.9992
Epoch: 276, Train Acc: 0.9992
Epoch: 277, Train Acc: 0.9992
Epoch: 278, Train Acc: 0.9992
Epoch: 279, Train Acc: 0.9992
Epoch: 280, Train Acc: 0.9992
Epoch: 281, Train Acc: 0.9992
Epoch: 282, Train Acc: 0.9992
Epoch: 283, Train Acc: 0.9992
Epoch: 284, Train Acc: 0.9992
Epoch: 285, Train Acc: 0.9992
Epoch: 286, Train Acc: 0.9992
Epoch: 287, Train Acc: 0.9992
Epoch: 288, Train Acc: 0.9992
Epoch: 289, Train Acc: 0.9992
Epoch: 290, Train Acc: 0.9990
Epoch: 291, Train Acc: 0.9992
Epoch: 292, Train Acc: 0.9992
Epoch: 293, Train Acc: 0.9992
Epoch: 294, Train Acc: 0.9992
Epoch: 295, Train Acc: 0.9992
Epoch: 296, Train Acc: 0.9992
Epoch: 297, Train Acc: 0.9992
Epoch: 298, Train Acc: 0.9992
Epoch: 299, Train Acc: 0.9992
Epoch: 300, Train Acc: 0.9989
Epoch: 301, Train Acc: 0.9917
Epoch: 302, Train Acc: 0.9967
Epoch: 303, Train Acc: 0.9989
Epoch: 304, Train Acc: 0.9992
Epoch: 305